## Czyszczenie danych

In [ ]:
select resultId, ra.name, ra.year, d.forename, d.surname, c.name, re.position, re.laps, s.status from results re
inner join races ra on ra.raceId = re.raceId
inner join drivers d on d.driverId = re.driverId
inner join constructors c on c.constructorId = re.constructorId
inner join status s on s.statusId = re.statusId
where laps <= 78

In [3]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Connect to SQLite database
db_name = 'f1.db'
connection = sqlite3.connect(db_name)

# Query the data from the table
query = 'select ra.name, ra.year, d.forename, d.surname, c.name, re.positionOrder, re.laps, s.status from results re inner join races ra on ra.raceId = re.raceId inner join drivers d on d.driverId = re.driverId inner join constructors c on c.constructorId = re.constructorId inner join status s on s.statusId = re.statusId'
result = connection.execute(query).fetchall()

# Get column names from the cursor description
columns = ["race_name", "race_year", "driver_firstname", "driver_lastname", "constructor_name", "driver_final_position", "laps_driven", "ending"]

# Create a DataFrame
df = pd.DataFrame(result, columns=columns)

# Close the connection
connection.close()

# Print the DataFrame
print(df)

                   race_name  race_year driver_firstname driver_lastname  \
0      Australian Grand Prix       2008            Lewis        Hamilton   
1      Australian Grand Prix       2008             Nick        Heidfeld   
2      Australian Grand Prix       2008             Nico         Rosberg   
3      Australian Grand Prix       2008         Fernando          Alonso   
4      Australian Grand Prix       2008           Heikki      Kovalainen   
...                      ...        ...              ...             ...   
26075     Belgian Grand Prix       2023           Daniel       Ricciardo   
26076     Belgian Grand Prix       2023            Logan        Sargeant   
26077     Belgian Grand Prix       2023             Nico     HĂĽlkenberg   
26078     Belgian Grand Prix       2023           Carlos           Sainz   
26079     Belgian Grand Prix       2023            Oscar         Piastri   

      constructor_name  driver_final_position  laps_driven            ending  
0       

## Średnia

In [ ]:
def mean(values):
    mean = 0
    for value in values:
        mean += value
    mean /= len(values)
    return mean

## Mediana

In [ ]:
def median(values):
    values.sort()
    n = len(values)
    if n % 2 == 1:
        return values[n // 2 + 1]
    else:
        return (values[n // 2] + values[n // 2 + 1]) / 2

## Rozstęp

In [ ]:
def sample_range(values):
    n = len(values)
    return values[n - 1] - values[0]

## Wariancja

In [8]:
def variance(values):
    n = len(values)
    m = mean(values)
    variance = 0
    for value in values:
        variance += (value - m) ** 2
    variance /= (n - 1)
    return variance

## Średnie odchylenie

In [ ]:
def average_deviation(values):
    n = len(values)
    deviation = 0
    m = mean(values)
    for value in values:
        deviation += abs(value - m)
    return deviation / n

## Dolny kwartyl

In [ ]:
def left_hinge(values):
    med = median(values)
    lower_values = []
    for value in values:
        if value <= med:
            lower_values.append(value)
    return median(lower_values)

## Górny kwartyl

In [ ]:
def right_hinge(values):
    med = median(values)
    upper_values = []
    for value in values:
        if value > med:
            upper_values.append(value)
    return median(upper_values)

In [ ]:
def print_info(values):
    print("Mean:\n", round(mean(values), 2))

    print("Median:\n", round(median(values.tolist()), 2))

    print("Range:\n:", round(sample_range(values), 2))

    print("Variance:\n", round(variance(values), 2))

    print("Standard deviation:\n", round(variance(values) ** 1 / 2, 2))

    print("Average deviation:\n", round(average_deviation(values), 2))

    print("Right hinge:\n", round(right_hinge(x_values.tolist()),2))

    print("Left hinge:\n", round(left_hinge(x_values.tolist()),2))

In [11]:
x_values = df["laps_driven"].astype(int)

In [ ]:
print_info(x_values)

## Histogram


In [10]:
plt.hist(x_values, bins=15)
plt.title("Histogram")
plt.xlabel("Laps driven")
plt.ylabel("Amount")
plt.show()

NameError: name 'x_values' is not defined

## Boxplot

In [ ]:
plt.boxplot(x_values)
plt.title("Boxplot")
plt.xlabel("Laps driven")
plt.show()

## Vioplot

In [ ]:
sns.violinplot(x_values)
plt.title("Violinplot")
plt.show()